# Produce plots for community structure

In [ ]:
%matplotlib notebook

import os
import utils
import sys
import numpy as np
from brainiak.utils import fmrisim as sim
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit
import colorsys

## Generate diagrams of community structure

In [ ]:
plt.figure()

# Make the base graph
graph = utils.community_structure(comm_spread=1,
                                 )

# Make and plot of the mid and low density
mid_density = utils.community_structure(comm_spread=0.5,
                                )
low_density = utils.community_structure(comm_spread=0,
                                )

# Make all the values positive and scale them to 1
mid_density = mid_density + abs(np.min(mid_density))
low_density = low_density + abs(np.min(low_density))
graph = graph + abs(np.min(graph))

mid_density = mid_density / np.max(mid_density)
low_density = low_density / np.max(low_density)
graph = graph / np.max(graph)

# Make random walk path
walk = np.asarray([0,2,4,1,3,0,3,4,5,7,9,6,8,5,8,9,10,12,14,11,13,10,13,14])  # A walk for all transitions
random_walk = graph[walk, :]
plt.plot(random_walk[:, 0], random_walk[:, 1], c=(0.9, 0.9, 0.9))

# Plot the nodes
plt.scatter(graph[:, 0], graph[:, 1], s=100, c='k', zorder=3)

hamilition = np.vstack([graph, graph[0,:]])
plt.plot(hamilition[:, 0], hamilition[:, 1], c=(0.9, 0.9, 0.9))

# Fix the values
plt.title('Community structure graph')
plt.axis('off')
plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])
plt.savefig('../../community_structure/plots/community_structure_graph.eps', format='eps', dpi=1000)

In [ ]:
# Make Hamilitonian path
plt.figure(figsize=(9,3))
high_d = np.vstack([graph, graph[0,:]])
plt.subplot(1, 3, 1)
plt.plot(high_d[:, 0], high_d[:, 1])
plt.axis('off')

mid_d = np.vstack([mid_density, mid_density[0,:]])
plt.subplot(1, 3, 2)
plt.plot(mid_d[:, 0], mid_d[:, 1])
plt.axis('off')

low_d = np.vstack([low_density, low_density[0,:]])
plt.subplot(1, 3, 3)
plt.plot(low_d[:, 0], low_d[:, 1])
plt.axis('off')

plt.savefig('../../community_structure/plots/community_structure_density.eps', format='eps', dpi=1000)

*Figure 1.* Graph depicting the community structure from Schapiro and colleagues (2013). The black circles represent each stimulus (fractals) participants were presented with. The blue and grey lines represent all of the possible transitions when taking a random walk, while the blue alone represents the hamilitonian path that participants took during the test trials. The graph here depicts when the density is 0 and the graph if the density is zero 

## Compare different signal parameters

In [ ]:
# Take the output of the signal_fit function and plot the heatmap of results

# Identify file paths
signal_fit_path='../../community_structure/signal_fit/'

# What is the file name
signal_fit_name = signal_fit_path + 'signal_fit.txt'

In [ ]:
# Load the noise parameters in
with open(signal_fit_name, 'r') as f:
    text = f.readlines()  # Pull out file as a an array

# Pull out the conditions and difference scores
sig_data = np.zeros([len(text), 3])
counter = 0
for line in text:
    condition, difference = line.strip().split()
    
    # Identify the condition variables
    density_idx = condition.find('density')
    signal_idx = condition.find('_s-')
    if density_idx > -1:
        density = float(condition[density_idx + 8:signal_idx])
    else:
        density = 1.0
    signal = float(condition[signal_idx + 3:-1])
    
    # Store the sig_data with the conditions
    sig_data[counter, :] = np.asarray([density, signal, float(difference)])
    
    # Increment counter
    counter += 1

In [ ]:
# Make a heat map of the results
densities = np.sort(np.unique(sig_data[:, 0]))
magnitudes = np.sort(np.unique(sig_data[:, 1]))
im_sig_data = np.ones([len(densities), len(magnitudes)]) * np.nan

# Cycle through density and magnitude
used_idx = []
for density_counter, density in enumerate(densities):
    for magnitude_counter, magnitude in enumerate(magnitudes):
        
        # Which index has this value
        den_idx = sig_data[:, 0] == density
        mag_idx = sig_data[:, 1] == magnitude
        
        # Which index is common between columns and rows
        idx = den_idx * mag_idx
        
        if np.any(idx):
            im_sig_data[density_counter, magnitude_counter] = sig_data[np.nonzero(idx), 2][0][0]
            
            used_idx.append(np.where(idx)[0][0])

# Rotate so that it is appropriately oriented
im_sig_data = np.flipud(np.rot90(im_sig_data));

In [ ]:
# Interpolate image sig_data so that nan's are ignored
X,Y=np.where(np.isnan(im_sig_data))

# Cycle through the nans
for nan_counter in list(range(len(X))):
    
    # Find the idxs surrounding
    X_min = X[nan_counter] - 1
    X_max = X[nan_counter] + 2
    Y_min = Y[nan_counter] - 1
    Y_max = Y[nan_counter] + 2
    
    # Bound
    if X_min < 0:
        X_min = 0
    if Y_min < 0:
        Y_min = 0
    if X_max > im_sig_data.shape[0]:
        X_max = im_sig_data.shape[0]
    if Y_max > im_sig_data.shape[1]:
        Y_max = im_sig_data.shape[1]
        
    # Make a mesh
    X_mesh, Y_mesh =np.meshgrid(np.arange(X_min, X_max), np.arange(Y_min, Y_max))
    
    # Insert the interpolated value
    im_sig_data[X[nan_counter], Y[nan_counter]] = np.nanmean(im_sig_data[X_mesh, Y_mesh])

In [ ]:
# Plot a 2d heat map of the difference from the result (be aware the signal is not evenly spaced)
plt.figure()
plt.imshow(np.flipud(abs(im_sig_data)))

# Set values
plt.title('Difference between real and simulated data')
plt.ylabel('Signal Magnitude')
plt.yticks(np.arange(0, len(magnitudes), 2), np.flipud(magnitudes)[::2])
#plt.tick_params(which='both', left='off', labelleft='off', bottom='off', labelbottom='off')
plt.xlabel('Density')
plt.xticks(np.arange(0, len(densities), 2), densities[::2])
plt.colorbar()
plt.clim((0,5))
plt.savefig('../../community_structure/plots/comparison_magnitude_density.eps', format='eps', dpi=100)

*Figure 2.* Heat map of the difference between the real and simulated data with different signal parameters. The Y axis is the signal magnitude (related to the percent signal change) and the X axis is the density of the community structure, where a density of zero is the blue outline in Figure 1 and a density of one is the orange triangle.

In [ ]:
# Plot the sig_data in 3d
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(sig_data[used_idx, 0], sig_data[used_idx, 1], abs(sig_data[used_idx, 2]))

X, Y=np.meshgrid(densities, magnitudes)
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, abs(im_sig_data))

# Set values
plt.title('Difference between real and simulated data')
ax.set_xlabel('Density')
ax.set_ylabel('Signal Magnitude')
ax.set_zlabel('Difference')

*Figure 3.* Similar to Figure 2 this depicts the difference between the real and simulated data with different signal parameters. The Y axis is the signal magnitude (related to the percent signal change) and the X axis is the density of the community structure, where a density of zero is the blue outline in Figure 1 and a density of one is the orange triangle.

In [ ]:
# Report the signal that is the minimum for each density
matched_signal = magnitudes[np.argmin(abs(im_sig_data),0)]
print('The signal magnitude that minimizes the difference between the real data and simulated data with different densities:')
print('Density:        ' + str(densities))
print('Matched Signal: ' + str(matched_signal))

In [ ]:
# Plot the line representing the transition in density

fig = plt.figure()
plt.scatter(densities, matched_signal)

# model = np.polyfit(x=np.log(density), y=Matched_Signal, deg=1)
# prediction = np.poly1d(model)
# trendline = prediction(density)

# Specify the function for an exponential
def exp_func(x, slope1, slope2, intercept):
    return slope1 * np.exp(-slope2 * x) + intercept

coefs, _ = curve_fit(exp_func, densities, matched_signal)
trendline = exp_func(densities, coefs[0], coefs[1], coefs[2])

plt.plot(densities, trendline)

density_levels = np.asarray([0.0, 0.5, 1.0])
signal_levels = np.round(exp_func(density_levels, coefs[0], coefs[1], coefs[2]) * 100) / 100  # Round to 2 DP

plt.title('Relationship between density and matched signal')
plt.ylabel('Matched Signal Magnitude')
plt.xlabel('Density')

print('What signal magnitude matches the real data at different amounts of density')
print('Density levels: ' + str(density_levels))
print('Matched Signal: ' + str(signal_levels))
plt.savefig('../../community_structure/plots/minimal_matched_signal_by_density.eps', format='eps', dpi=100)

*Figure 4.* What are the values of density and signal magnitude that minimize the difference between the real and simulated data. The Y axis is the signal magnitude that minimizes the difference between the real and simulated data for a given density and the X axis is the density of the community structure.

## Show the effects of different delays

In [ ]:
# Create a function to take in a list of onsets and output a plot with the onsets and hrfs overlaid on each other
def plot_onsets(onsets,
               ):
    tr_duration=2
    temporal_resolution=1
    starting_idx = 50 * temporal_resolution
    ending_idx = 200 * temporal_resolution
    min_hue = 50 / 360
    max_hue = 220 /360
    hue_steps = np.linspace(min_hue, max_hue, 15)
    
    # Get coordinate values to use
    graph = utils.community_structure(comm_spread=1,
                                 )
    graph = graph + abs(graph.min())
    for node_counter in list(range(0, 15)):
        # Create the time course for the signal to be generated
        stimfunc = sim.generate_stimfunction(onsets=onsets[node_counter],
                                             event_durations=[1],
                                             total_time=1000,
                                             temporal_resolution=temporal_resolution,
                                             )

        # Only take the first block of trials 
        stimfunc = stimfunc[starting_idx:ending_idx]
        
        # Multiply the coordinate value
        stimfunc *= graph[node_counter, 0]
        
        # Add the stim funcs together
        if node_counter == 0:
            stimfunc_all = stimfunc
        else:
            stimfunc_all += stimfunc
        
        # Interpolate colors allowing a hue

        rgb = colorsys.hsv_to_rgb(hue_steps[node_counter], 0.75, 1)
        
        # Plot each individual event
        plt.plot(stimfunc, c=rgb)

    # Convolve all of the events with the hrf
    hrf = sim.convolve_hrf(stimfunction=stimfunc_all, 
                           tr_duration=tr_duration, 
                           temporal_resolution=temporal_resolution,
                          )
    
    # Upsample the hrf to be in seconds (rather than trs)
    hrf_interpolated = np.zeros((stimfunc.shape[0],))
    for tr_counter in list(range(len(hrf) - 1)):
        hrf_interpolated[tr_counter * 2] = hrf[tr_counter]
        hrf_interpolated[(tr_counter * 2) + 1] = np.mean([hrf[tr_counter], hrf[tr_counter + 1]])
    hrf_interpolated[-1] = hrf[-1]
    
    # Plot the interpolated hrf
    plt.plot(hrf_interpolated, c='g')

In [ ]:
# Load an example of Anna's dataset and show the effects of different ISIs and randomization

# Note, there should only be 12 events because we ignore the first 4

signal_fit_path = '../../community_structure/simulator_parameters/timing/sub-1.npy'

# Plot the default values
onsets = np.load(signal_fit_path)[0]
plt.figure()

plt.subplot(2, 2, 1)
plot_onsets(onsets)
plt.title('Minimum ISI:1s')
plt.ylabel('Hamilitonian\nEstimated Response')
plt.ylim([0, 5])

plt.subplot(2, 2, 2)
onsets = np.load(signal_fit_path)[0]
plot_onsets(utils.extra_isi(onsets, 5))
plt.title('Minimum ISI: 6s')
plt.ylim([0, 5])

plt.subplot(2, 2, 3)
onsets = np.load(signal_fit_path)[0]
plot_onsets(utils.randomise_timing(onsets))
plt.ylabel('Randomized\nEstimated Response')
plt.xlabel('Added ISI (s)')
plt.ylim([0, 5])

plt.subplot(2, 2, 4)
onsets = np.load(signal_fit_path)[0]
plot_onsets(utils.randomise_timing(utils.extra_isi(onsets, 5)))
plt.xlabel('Added ISI (s)')
plt.ylim([0, 5])
plt.savefig('../../community_structure/plots/altered_event_order.eps', format='eps', dpi=100)

*Figure 5.* Estimated response for a single voxel under different experimental designs. The dark green line represents the convolution of the color events with a double gamma HRF. The events are colored across a hue spectrum from teal to blue as you traverse the hamilitonian path

## Compare different delay values with a toy simulation

In [ ]:
# Compare different density and ISI values
r_min = -0.5
r_max = 1
plt.figure()
plt.subplot(2, 2, 1)
node_brain = utils.toy_simulation(community_density=1, 
                                  added_isi=0, 
                                  rand=0,
                                  )
plt.scatter(node_brain[0, :], node_brain[1, :])
plt.xlim([r_min, r_max]); plt.ylim([r_min, r_max])
plt.xticks([]); plt.yticks([])
plt.title('Added ISI = 0, Hamiliton')
plt.ylabel('Density = 1')

plt.subplot(2, 2, 2)
node_brain = utils.toy_simulation(community_density=1, 
                                  added_isi=5, 
                                  rand=0,
                                  )
plt.scatter(node_brain[0, :], node_brain[1, :])
plt.xlim([r_min, r_max]); plt.ylim([r_min, r_max])
plt.xticks([]); plt.yticks([])
plt.title('Added ISI = 5, Random')

plt.subplot(2, 2, 3)
node_brain = utils.toy_simulation(community_density=0, 
                                  added_isi=0, 
                                  rand=0,
                                  )
plt.scatter(node_brain[0, :], node_brain[1, :])
plt.xlim([r_min, r_max]); plt.ylim([r_min, r_max])
plt.xticks([]); plt.yticks([])
plt.ylabel('Density = 0')

plt.subplot(2, 2, 4)
node_brain = utils.toy_simulation(community_density=0, 
                                  added_isi=5, 
                                  rand=0,
                                  )
plt.scatter(node_brain[0, :], node_brain[1, :])
plt.xlim([r_min, r_max]); plt.ylim([r_min, r_max])
plt.xticks([]); plt.yticks([])
plt.savefig('../../community_structure/plots/density_by_event_timing_representation.eps', format='eps', dpi=100)

In [ ]:
# Make toy data in order to estimate the trends this way.

limit = 0
max_delay = 10
toy_data = np.zeros((max_delay, 3, 2))
toy_signal_levels = np.asarray([3, 1.45, 0.9])
density_levels = [0.0, 0.5, 1.0]
for density_counter, density in enumerate(density_levels):
    signal = toy_signal_levels[density_counter]
    for added_isi in list(range(0, max_delay)):
        for rand in list(range(2)):
            vol = utils.toy_simulation(community_density=density, 
                                              added_isi=added_isi, 
                                              rand=rand,
                                              signal_magnitude=signal,
                                              restrict_overall_duration=limit,
                                              )
            # Run the analysis
            toy_data[added_isi, density_counter, rand] = utils.test_rsa(np.transpose(vol),
                                                                         distance_type='distance',
                                                                         )
            
plt.figure()
plt.subplot(1, 2, 1)
plt.plot(toy_data[:, :, 0])
plt.ylim((0, 1))
plt.ylabel('Within vs Between distance')
plt.xlabel('Added ISI (s)')
plt.title('Hamiltonian')

plt.subplot(1, 2, 2)
plt.plot(toy_data[:, :, 1])
plt.ylim((0, 1))
plt.legend(density_levels, title='Density')
plt.xlabel('Added ISI (s)')
plt.title('Randomise')
plt.savefig('../../community_structure/plots/toy_delay.eps', format='eps', dpi=100)

In [ ]:
limit = 1
max_delay = 10
toy_data = np.zeros((max_delay, 3, 2))
toy_signal_levels = np.asarray([3, 1.45, 0.9])
density_levels = [0.0, 0.5, 1.0]
for density_counter, density in enumerate(density_levels):
    signal = toy_signal_levels[density_counter]
    for added_isi in list(range(0, max_delay)):
        for rand in list(range(1)):
            vol = utils.toy_simulation(community_density=density, 
                                              added_isi=added_isi, 
                                              rand=rand,
                                              signal_magnitude=signal,
                                              restrict_overall_duration=limit,
                                              )
            # Run the analysis
            toy_data[added_isi, density_counter, rand] = utils.test_rsa(np.transpose(vol),
                                                                        distance_type='distance',
                                                                       )

plt.figure()
plt.subplot(1, 2, 1)
plt.plot(toy_data[:, :, 0])
plt.ylim((0, 1))
plt.ylabel('Within vs Between distance')
plt.xlabel('Added ISI (s)')
plt.title('Limited Hamiltonian')

plt.savefig('../../community_structure/plots/toy_delay_limited.eps', format='eps', dpi=100)

## Compare different delay values with a realistic simulation

In [ ]:
# Find the file names
delay_file = '../../community_structure/delay/delay.txt'
delay_max = 11  # What is the max delay idx you want to show

# Load the noise parameters in
with open(delay_file, 'r') as f:
    text = f.readlines()  # Pull out file as a an array

In [ ]:
# Pull out the conditions and difference scores
delay_data = {}
counter = 0
for line in text:
    condition, difference = line.strip().split()
    
    # Identify the condition variables
    delay_idx = condition.find('_t-')
    randomise_idx = condition.find('_r-')
    density_idx = condition.find('density')
    signal_idx = condition.find('_s-')
    limit_idx = condition.find('limit') 
    resample_idx = condition.find('_resample-') 
    

    # Identify the condition variables
    delay = int(condition[delay_idx + 3:randomise_idx])
    randomise = int(condition[randomise_idx + 3:randomise_idx + 4])
    if density_idx > -1:
        density = float(condition[density_idx + 8:signal_idx])
    else:
        density = 1.0
    signal = condition[signal_idx+ 3:signal_idx + 7]
    if signal[-1] == '_':
        signal = signal[:-1]
    
    if limit_idx > -1:
        limit = 1
    else:
        limit = 0
    resample = int(condition[resample_idx + 10:-1])
    
    # Store the sig_data with the conditions
    summary_condition = 'r-%d_d-%0.1f_s-%s_l-%d' % (randomise, density, signal, limit)
    
    # Preset the array if it doesn't exist
    if summary_condition not in delay_data:
        delay_data[summary_condition] = np.ones((16, 10)) * np.nan
    
    # Store the data
    delay_data[summary_condition][delay, resample - 1] = float(difference)
    

In [ ]:
# Summarize the data you have
print('Check how many resamples per condition there are')
for key in delay_data:
    print(key)
    print(np.sum(~np.isnan(delay_data[key])[:delay_max], 1))


In [ ]:
low_density_sig = '0.5'
mid_density_sig = '0.35'
high_density_sig = '0.25'

In [ ]:
# Plot the data along with error

def plot_delay_data(delay_data, keys, delay_max):
    plt.ylim([-2, 5])
    plt.hlines(0, 0, delay_max, linestyles='dashed')
    
    # Iterate through all the listed keys
    for key in keys:
        y = delay_data[key][:delay_max, :]
        #err = (np.nanstd(y, 1) / np.sqrt(np.sum(~np.isnan(y), 1)))  # Standard error
        err = np.nanstd(y, 1)  # Standard deviation
        plt.plot(np.arange(delay_max), np.nanmean(y, 1))
        plt.fill_between(np.arange(delay_max), np.nanmean(y, 1) + err, np.nanmean(y, 1) - err, alpha=0.5)
    

In [ ]:
# Plot the delay at different density levels
plt.figure()

# Plot the hamilitonian data
plt.subplot(1,3,1)

keys = ['r-0_d-0.0_s-%s_l-0' % low_density_sig, 'r-0_d-0.5_s-%s_l-0' % mid_density_sig, 'r-0_d-1.0_s-%s_l-0' % high_density_sig]
plot_delay_data(delay_data, keys, delay_max)
plt.title('A')
plt.ylabel('Diff. t stat in ROI')
plt.title('Hamiltonian')

# Plot the randomised data
plt.subplot(1,3,2)
keys = ['r-1_d-0.0_s-%s_l-0' % low_density_sig, 'r-1_d-0.5_s-%s_l-0' % mid_density_sig, 'r-1_d-1.0_s-%s_l-0' % high_density_sig]
plot_delay_data(delay_data, keys, delay_max)
plt.title('B')
plt.xlabel('Added ISI (s)')
plt.title('Random')

# Plot the hamilitonian data with the time limit
plt.subplot(1,3,3)
keys = ['r-0_d-0.0_s-%s_l-1' % low_density_sig, 'r-0_d-0.5_s-%s_l-1' % mid_density_sig, 'r-0_d-1.0_s-%s_l-1' % high_density_sig]
plot_delay_data(delay_data, keys, delay_max)
plt.title('C')
plt.legend(['0.0', '0.5', '1.0', 'real result'], title='Density')
plt.title('Hamiltonian')

plt.savefig('../../community_structure/plots/simulated_delay.eps', format='eps', dpi=100)

*Figure 6.* Average t statistic in ROI for different amounts of ISI from 10 simulations of the data. A) Events are presented in a hamiltonian path. B) Events are presented in a random order. C) Number of events are limited by the expeiment duration. Different lines represent different densities of the simulated community structure. The black line represents the real t statistic, shaded lines are standard deviation across resamples.